In [1]:
# ============================================
# KAGGLE SETUP - RUN THIS FIRST
# ============================================

print("🚀 Setting up Trading Backtest Engine on Kaggle...")
print("=" * 60)

# Install additional packages (most are pre-installed)
!pip install -q yfinance pandas-ta loguru

print("✅ Packages installed!")

# Create directory structure
import os
os.makedirs('src/backtesting', exist_ok=True)
os.makedirs('src/data_ingestion', exist_ok=True)
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/results', exist_ok=True)

print("✅ Directories created!")
print("=" * 60)

import warnings
warnings.filterwarnings('ignore')

print("\n🎉 Kaggle environment ready!")
print(f"💪 Available RAM: ~30GB (vs Colab's 12GB)")
print("📊 Ready to handle 100+ stocks!")

🚀 Setting up Trading Backtest Engine on Kaggle...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 60.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 56.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 96.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 39.8 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
ydata

In [3]:
#Cell 2
# ============================================
# CREATE STRATEGY GRID GENERATOR
# ============================================

strategy_code = '''
from typing import List, Dict
from itertools import product

class StrategyGridGenerator:
    
    @staticmethod
    def generate_all_strategies() -> List[Dict]:
        """Generate 200+ strategies"""
        strategies = []
        
        # Moving Average: ~30 combinations
        short_windows = [5, 10, 15, 20, 25, 30, 40, 50]
        long_windows = [50, 100, 150, 200]
        for short, long in product(short_windows, long_windows):
            if short < long:
                strategies.append({
                    'type': 'moving_average',
                    'short_window': short,
                    'long_window': long,
                    'name': f'MA_{short}_{long}'
                })
        
        # RSI: 45 combinations
        periods = [7, 9, 14, 21, 28]
        oversold_levels = [20, 25, 30]
        overbought_levels = [70, 75, 80]
        for period in periods:
            for oversold in oversold_levels:
                for overbought in overbought_levels:
                    strategies.append({
                        'type': 'rsi',
                        'period': period,
                        'oversold': oversold,
                        'overbought': overbought,
                        'name': f'RSI_{period}_{oversold}_{overbought}'
                    })
        
        # MACD: 27 combinations
        fast_periods = [8, 12, 16]
        slow_periods = [20, 26, 32]
        signal_periods = [7, 9, 11]
        for fast in fast_periods:
            for slow in slow_periods:
                for signal in signal_periods:
                    if fast < slow:
                        strategies.append({
                            'type': 'macd',
                            'fast_period': fast,
                            'slow_period': slow,
                            'signal_period': signal,
                            'name': f'MACD_{fast}_{slow}_{signal}'
                        })
        
        # Mean Reversion: 20 combinations
        windows = [10, 15, 20, 25, 30]
        std_multiples = [1.5, 2.0, 2.5, 3.0]
        for window in windows:
            for std_mult in std_multiples:
                strategies.append({
                    'type': 'mean_reversion',
                    'window': window,
                    'num_std': std_mult,
                    'name': f'BB_{window}_{std_mult}'
                })
        
        return strategies
'''

with open('src/backtesting/strategy_grid.py', 'w') as f:
    f.write(strategy_code)

print("✅ Created: strategy_grid.py")

# Test it
exec(strategy_code)
test_strategies = StrategyGridGenerator.generate_all_strategies()
print(f"✅ Generated {len(test_strategies)} strategies")

✅ Created: strategy_grid.py
✅ Generated 123 strategies


In [4]:
#Cell 3
# ============================================
# DOWNLOAD 100 STOCKS OF REAL DATA
# ============================================

import yfinance as yf
import pandas as pd
from datetime import datetime
import time

print("📊 DOWNLOADING 100 STOCKS - MAXIMUM SCALE!")
print("=" * 60)

# KAGGLE CAN HANDLE THIS!
NUM_STOCKS = 100             # ⬆️⬆️⬆️ 100 stocks!
START_DATE = "2014-01-01"    # 10 years
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Top 100 most liquid stocks
TICKERS_100 = [
    # ETFs
    'SPY', 'QQQ', 'IWM', 'DIA', 'VTI', 'EFA', 'EEM', 'AGG', 'GLD', 'TLT',
    # Mega Cap
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'BRK-B', 'AVGO', 'LLY',
    # Tech
    'ORCL', 'ADBE', 'CRM', 'AMD', 'INTC', 'CSCO', 'NFLX', 'QCOM', 'TXN', 'INTU',
    # Finance
    'JPM', 'BAC', 'WFC', 'GS', 'MS', 'V', 'MA', 'AXP', 'BLK', 'C', 'SCHW', 'USB', 'PNC', 'TFC', 'COF',
    # Healthcare
    'UNH', 'JNJ', 'ABBV', 'MRK', 'TMO', 'ABT', 'PFE', 'DHR', 'BMY', 'CVS', 'ELV', 'CI', 'HUM', 'AMGN', 'GILD',
    # Consumer
    'WMT', 'PG', 'COST', 'HD', 'MCD', 'NKE', 'DIS', 'SBUX', 'TGT', 'LOW', 'KO', 'PEP', 'PM', 'MDLZ', 'CL',
    # Industrial
    'CAT', 'HON', 'UPS', 'BA', 'RTX', 'LMT', 'DE', 'GE', 'MMM', 'FDX',
    # Energy
    'XOM', 'CVX', 'COP', 'SLB', 'EOG', 'MPC', 'PSX', 'VLO', 'OXY', 'HAL',
    # Other
    'NEE', 'DUK', 'SO', 'T', 'VZ', 'CMCSA', 'NEM', 'FCX', 'APD', 'LIN'
]

print(f"Configuration:")
print(f"  • Stocks: {NUM_STOCKS} ⬆️⬆️⬆️")
print(f"  • Years: 10")
print(f"  • Expected records: ~{NUM_STOCKS * 2520:,}")
print("\n" + "=" * 60)

# Download in batches (more reliable)
print("\n🚀 Downloading in batches...\n")

batch_size = 20
all_data = []

for batch_num in range(0, NUM_STOCKS, batch_size):
    batch_tickers = TICKERS_100[batch_num:batch_num + batch_size]
    print(f"\n📥 Batch {batch_num//batch_size + 1}: {', '.join(batch_tickers)}")
    
    try:
        batch_str = ' '.join(batch_tickers)
        data = yf.download(batch_str, start=START_DATE, end=END_DATE, group_by='ticker', threads=True)
        
        for ticker in batch_tickers:
            try:
                if len(batch_tickers) == 1:
                    df = data.reset_index()
                else:
                    df = data[ticker].reset_index()
                
                df.columns = df.columns.str.lower()
                df['ticker'] = ticker
                df = df[['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']].copy()
                df.rename(columns={'date': 'timestamp'}, inplace=True)
                df['timestamp'] = pd.to_datetime(df['timestamp'])
                df = df.dropna()
                
                if len(df) > 100:
                    all_data.append(df)
                    print(f"  ✅ {ticker}: {len(df):,} rows")
            except:
                print(f"  ❌ {ticker}")
        
        time.sleep(2)  # Be nice to API
        
    except Exception as e:
        print(f"  ⚠️ Batch failed: {str(e)[:50]}")

# Combine
df = pd.concat(all_data, ignore_index=True)

print("\n" + "=" * 60)
print("✅ DOWNLOAD COMPLETE!")
print("=" * 60)
print(f"\n🎯 MAXIMUM SCALE Achieved:")
print(f"  • Total records: {len(df):,} ⬆️⬆️⬆️")
print(f"  • Successfully downloaded: {len(all_data)} stocks")
print(f"  • Date range: {df['timestamp'].min().date()} to {df['timestamp'].max().date()}")
print(f"  • Years: {(df['timestamp'].max() - df['timestamp'].min()).days / 365:.1f}")

# Save
df.to_parquet('data/raw/sp500_prices.parquet', index=False)
print(f"\n✅ Saved {len(df):,} records!")
print("=" * 60)

📊 DOWNLOADING 100 STOCKS - MAXIMUM SCALE!
Configuration:
  • Stocks: 100 ⬆️⬆️⬆️
  • Years: 10
  • Expected records: ~252,000


🚀 Downloading in batches...


📥 Batch 1: SPY, QQQ, IWM, DIA, VTI, EFA, EEM, AGG, GLD, TLT, AAPL, MSFT, GOOGL, AMZN, NVDA, META, TSLA, BRK-B, AVGO, LLY


/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[*******               15%                       ]  3 of 20 completed/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[**********            20%                       ]  4 of 20 completed/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[*****************     35%                       ]  7 of 20 completed/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow

  ✅ SPY: 3,036 rows
  ✅ QQQ: 3,036 rows
  ✅ IWM: 3,036 rows
  ✅ DIA: 3,036 rows
  ✅ VTI: 3,036 rows
  ✅ EFA: 3,036 rows
  ✅ EEM: 3,036 rows
  ✅ AGG: 3,036 rows
  ✅ GLD: 3,036 rows
  ✅ TLT: 3,036 rows
  ✅ AAPL: 3,036 rows
  ✅ MSFT: 3,036 rows
  ✅ GOOGL: 3,036 rows
  ✅ AMZN: 3,036 rows
  ✅ NVDA: 3,036 rows
  ✅ META: 3,036 rows
  ✅ TSLA: 3,036 rows
  ✅ BRK-B: 3,036 rows
  ✅ AVGO: 3,036 rows
  ✅ LLY: 3,036 rows

📥 Batch 2: ORCL, ADBE, CRM, AMD, INTC, CSCO, NFLX, QCOM, TXN, INTU, JPM, BAC, WFC, GS, MS, V, MA, AXP, BLK, C


/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[                       0%                       ]/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[*******               15%                       ]  3 of 20 completed/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[**********            20%                       ]  4 of 20 completed/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and 

  ✅ ORCL: 3,036 rows
  ✅ ADBE: 3,036 rows
  ✅ CRM: 3,036 rows
  ✅ AMD: 3,036 rows
  ✅ INTC: 3,036 rows
  ✅ CSCO: 3,036 rows
  ✅ NFLX: 3,036 rows
  ✅ QCOM: 3,036 rows
  ✅ TXN: 3,036 rows
  ✅ INTU: 3,036 rows
  ✅ JPM: 3,036 rows
  ✅ BAC: 3,036 rows
  ✅ WFC: 3,036 rows
  ✅ GS: 3,036 rows
  ✅ MS: 3,036 rows
  ✅ V: 3,036 rows
  ✅ MA: 3,036 rows
  ✅ AXP: 3,036 rows
  ✅ BLK: 3,036 rows
  ✅ C: 3,036 rows

📥 Batch 3: SCHW, USB, PNC, TFC, COF, UNH, JNJ, ABBV, MRK, TMO, ABT, PFE, DHR, BMY, CVS, ELV, CI, HUM, AMGN, GILD


...

  ✅ SCHW: 3,036 rows
  ✅ USB: 3,036 rows
  ✅ PNC: 3,036 rows
  ✅ TFC: 3,036 rows
  ✅ COF: 3,036 rows
  ✅ UNH: 3,036 rows
  ✅ JNJ: 3,036 rows
  ✅ ABBV: 3,036 rows
  ✅ MRK: 3,036 rows
  ✅ TMO: 3,036 rows
  ✅ ABT: 3,036 rows
  ✅ PFE: 3,036 rows
  ✅ DHR: 3,036 rows
  ✅ BMY: 3,036 rows
  ✅ CVS: 3,036 rows
  ✅ ELV: 3,036 rows
  ✅ CI: 3,036 rows
  ✅ HUM: 3,036 rows
  ✅ AMGN: 3,036 rows
  ✅ GILD: 3,036 rows

📥 Batch 4: WMT, PG, COST, HD, MCD, NKE, DIS, SBUX, TGT, LOW, KO, PEP, PM, MDLZ, CL, CAT, HON, UPS, BA, RTX


/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[                       0%                       ]/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[*******               15%                       ]  3 of 20 completed/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead

  ✅ WMT: 3,036 rows
  ✅ PG: 3,036 rows
  ✅ COST: 3,036 rows
  ✅ HD: 3,036 rows
  ✅ MCD: 3,036 rows
  ✅ NKE: 3,036 rows
  ✅ DIS: 3,036 rows
  ✅ SBUX: 3,036 rows
  ✅ TGT: 3,036 rows
  ✅ LOW: 3,036 rows
  ✅ KO: 3,036 rows
  ✅ PEP: 3,036 rows
  ✅ PM: 3,036 rows
  ✅ MDLZ: 3,036 rows
  ✅ CL: 3,036 rows
  ✅ CAT: 3,036 rows
  ✅ HON: 3,036 rows
  ✅ UPS: 3,036 rows
  ✅ BA: 3,036 rows
  ✅ RTX: 3,036 rows

📥 Batch 5: LMT, DE, GE, MMM, FDX, XOM, CVX, COP, SLB, EOG, MPC, PSX, VLO, OXY, HAL, NEE, DUK, SO, T, VZ


/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[*****                 10%                       ]  2 of 20 completed/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[*******               15%                       ]  3 of 20 completed/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp

  ✅ LMT: 3,036 rows
  ✅ DE: 3,036 rows
  ✅ GE: 3,036 rows
  ✅ MMM: 3,036 rows
  ✅ FDX: 3,036 rows
  ✅ XOM: 3,036 rows
  ✅ CVX: 3,036 rows
  ✅ COP: 3,036 rows
  ✅ SLB: 3,036 rows
  ✅ EOG: 3,036 rows
  ✅ MPC: 3,036 rows
  ✅ PSX: 3,036 rows
  ✅ VLO: 3,036 rows
  ✅ OXY: 3,036 rows
  ✅ HAL: 3,036 rows
  ✅ NEE: 3,036 rows
  ✅ DUK: 3,036 rows
  ✅ SO: 3,036 rows
  ✅ T: 3,036 rows
  ✅ VZ: 3,036 rows

✅ DOWNLOAD COMPLETE!

🎯 MAXIMUM SCALE Achieved:
  • Total records: 303,600 ⬆️⬆️⬆️
  • Successfully downloaded: 100 stocks
  • Date range: 2014-01-02 to 2026-01-28
  • Years: 12.1

✅ Saved 303,600 records!


In [5]:
#Cell 4
# ============================================
# INITIALIZE SPARK FOR BIG DATA
# ============================================

from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')

print("🚀 Initializing PySpark for MAXIMUM SCALE...")
print("=" * 60)

# Stop previous session
try:
    spark.stop()
except:
    pass

# Create Spark with max memory (Kaggle has 30GB!)
spark = SparkSession.builder \
    .appName("TradingBacktest_MaxScale") \
    .master("local[*]") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.driver.maxResultSize", "4g") \
    .getOrCreate()

print(f"✅ Spark initialized with MAXIMUM memory!")
print(f"   Version: {spark.version}")
print(f"   Memory: 12GB driver + 12GB executor")

# Load big dataset
print("\n📂 Loading SCALED data...")
price_data = spark.read.parquet("data/raw/sp500_prices.parquet")

num_records = price_data.count()
num_tickers = price_data.select("ticker").distinct().count()

print(f"\n✅ BIG DATA loaded!")
print(f"   Records: {num_records:,} ⬆️⬆️⬆️")
print(f"   Tickers: {num_tickers} ⬆️⬆️⬆️")
print(f"   Scale: {num_records/17780:.1f}x larger than original!")

print("\n📊 Sample:")
price_data.show(5)

print("\n" + "=" * 60)
print("✅ Ready for MAXIMUM SCALE backtesting!")
print("=" * 60)

🚀 Initializing PySpark for MAXIMUM SCALE...


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/29 04:10:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark initialized with MAXIMUM memory!
   Version: 3.5.1
   Memory: 12GB driver + 12GB executor

📂 Loading SCALED data...



✅ BIG DATA loaded!
   Records: 303,600 ⬆️⬆️⬆️
   Tickers: 100 ⬆️⬆️⬆️
   Scale: 17.1x larger than original!

📊 Sample:
+------+-------------------+------------------+------------------+------------------+------------------+---------+
|ticker|          timestamp|              open|              high|               low|             close|   volume|
+------+-------------------+------------------+------------------+------------------+------------------+---------+
|   SPY|2014-01-02 00:00:00|149.84942358396367|149.92273691254042|148.62769219687948|148.98606872558594|119636900|
|   SPY|2014-01-03 00:00:00|149.23850106202124|149.53986980718156| 148.7498161501301|148.96157836914062| 81390600|
|   SPY|2014-01-06 00:00:00|149.45031257597307| 149.5073202371162|148.30188224338275|148.52993774414062|108028200|
|   SPY|2014-01-07 00:00:00|149.12450432210807|149.69464305181214|149.01047657616726|149.44215393066406| 86144200|
|   SPY|2014-01-08 00:00:00| 149.4177418676441|149.72725112048178|148.961630

In [6]:
#Cell 5
# ============================================
# GENERATE FULL STRATEGY GRID
# ============================================

import sys
sys.path.insert(0, 'src')

exec(open('src/backtesting/strategy_grid.py').read())

print("🎯 GENERATING FULL STRATEGY GRID")
print("=" * 60)

strategies = StrategyGridGenerator.generate_all_strategies()

print(f"\n✅ Generated {len(strategies)} strategies ⬆️⬆️⬆️")

from collections import Counter
strategy_types = Counter(s['type'] for s in strategies)
print(f"\nStrategy breakdown:")
for stype, count in strategy_types.items():
    print(f"  • {stype}: {count} strategies")

# Calculate total backtests
total_backtests = len(strategies) * num_tickers
print(f"\n📊 MAXIMUM SCALE:")
print(f"   Total backtests to run: {total_backtests:,} ⬆️⬆️⬆️")
print(f"   ({len(strategies)} strategies × {num_tickers} stocks)")
print(f"   vs Original Colab: {total_backtests/3690:.1f}x MORE backtests!")

print(f"\n⏱️ Estimated runtime: 30-60 minutes")
print(f"💡 Kaggle allows up to 9 hours - plenty of time!")
print("=" * 60)

🎯 GENERATING FULL STRATEGY GRID

✅ Generated 123 strategies ⬆️⬆️⬆️

Strategy breakdown:
  • moving_average: 31 strategies
  • rsi: 45 strategies
  • macd: 27 strategies
  • mean_reversion: 20 strategies

📊 MAXIMUM SCALE:
   Total backtests to run: 12,300 ⬆️⬆️⬆️
   (123 strategies × 100 stocks)
   vs Original Colab: 3.3x MORE backtests!

⏱️ Estimated runtime: 30-60 minutes
💡 Kaggle allows up to 9 hours - plenty of time!


In [8]:
#Cell 6
# ============================================
# RUN MAXIMUM SCALE BACKTEST
# ============================================

from pyspark.sql.functions import col, avg, stddev, min as spark_min, max as spark_max, count
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
import pandas as pd
import numpy as np
from datetime import datetime
import json

print("=" * 80)
print("🚀 MAXIMUM SCALE PARALLEL BACKTESTING")
print("=" * 80)
print(f"Processing {total_backtests:,} backtests...")
print("This is 6-10x MORE than your Colab run!")
print("=" * 80)

# Define backtest function
def backtest_udf(pdf: pd.DataFrame) -> pd.DataFrame:
    import pandas as pd
    import numpy as np
    import ast
    
    strategy_params = ast.literal_eval(pdf['strategy_params'].iloc[0])
    strategy_type = pdf['strategy_type'].iloc[0]
    strategy_id = pdf['strategy_id'].iloc[0]
    ticker = pdf['ticker'].iloc[0]
    
    pdf = pdf.sort_values('timestamp')
    
    # Generate signals
    if strategy_type == "moving_average":
        short_window = strategy_params.get('short_window', 20)
        long_window = strategy_params.get('long_window', 50)
        ma_short = pdf['close'].rolling(window=short_window).mean()
        ma_long = pdf['close'].rolling(window=long_window).mean()
        signals = pd.Series(0, index=pdf.index)
        signals[ma_short > ma_long] = 1
        signals[ma_short < ma_long] = -1
        
    elif strategy_type == "rsi":
        period = strategy_params.get('period', 14)
        oversold = strategy_params.get('oversold', 30)
        overbought = strategy_params.get('overbought', 70)
        delta = pdf['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / (loss + 1e-10)
        rsi = 100 - (100 / (1 + rs))
        signals = pd.Series(0, index=pdf.index)
        signals[rsi < oversold] = 1
        signals[rsi > overbought] = -1
        
    elif strategy_type == "macd":
        fast = strategy_params.get('fast_period', 12)
        slow = strategy_params.get('slow_period', 26)
        signal_period = strategy_params.get('signal_period', 9)
        ema_fast = pdf['close'].ewm(span=fast).mean()
        ema_slow = pdf['close'].ewm(span=slow).mean()
        macd_line = ema_fast - ema_slow
        signal_line = macd_line.ewm(span=signal_period).mean()
        signals = pd.Series(0, index=pdf.index)
        signals[macd_line > signal_line] = 1
        signals[macd_line < signal_line] = -1
        
    elif strategy_type == "mean_reversion":
        window = strategy_params.get('window', 20)
        num_std = strategy_params.get('num_std', 2)
        rolling_mean = pdf['close'].rolling(window=window).mean()
        rolling_std = pdf['close'].rolling(window=window).std()
        upper_band = rolling_mean + (rolling_std * num_std)
        lower_band = rolling_mean - (rolling_std * num_std)
        signals = pd.Series(0, index=pdf.index)
        signals[pdf['close'] < lower_band] = 1
        signals[pdf['close'] > upper_band] = -1
    else:
        signals = pd.Series(0, index=pdf.index)
    
    # Calculate returns
    pdf['returns'] = pdf['close'].pct_change()
    pdf['strategy_returns'] = signals.shift(1) * pdf['returns']
    returns_clean = pdf['strategy_returns'].dropna()
    
    # Calculate metrics
    if len(returns_clean) == 0 or returns_clean.std() == 0:
        metrics = {'sharpe_ratio': 0.0, 'max_drawdown': 0.0, 'total_return': 0.0,
                   'win_rate': 0.0, 'calmar_ratio': 0.0, 'num_trades': 0, 'avg_trade_return': 0.0}
    else:
        sharpe = np.sqrt(252) * returns_clean.mean() / returns_clean.std()
        cumulative = (1 + returns_clean).cumprod()
        drawdown = ((cumulative - cumulative.expanding().max()) / cumulative.expanding().max()).min()
        total_ret = (1 + returns_clean).prod() - 1
        win_rate = (returns_clean > 0).sum() / len(returns_clean)
        annual_ret = (1 + total_ret) ** (252 / len(returns_clean)) - 1
        calmar = annual_ret / abs(drawdown) if drawdown != 0 else 0
        
        metrics = {
            'sharpe_ratio': float(sharpe) if not np.isnan(sharpe) else 0.0,
            'max_drawdown': float(drawdown) if not np.isnan(drawdown) else 0.0,
            'total_return': float(total_ret) if not np.isnan(total_ret) else 0.0,
            'win_rate': float(win_rate),
            'calmar_ratio': float(calmar) if not np.isnan(calmar) else 0.0,
            'num_trades': int(len(returns_clean[returns_clean != 0])),
            'avg_trade_return': float(returns_clean[returns_clean != 0].mean()) if len(returns_clean[returns_clean != 0]) > 0 else 0.0
        }
    
    return pd.DataFrame([{'ticker': ticker, 'strategy_id': strategy_id, 
                          'strategy_type': strategy_type, 'strategy_params': str(strategy_params), **metrics}])

# Prepare strategy DataFrame
strategy_rows = [{"strategy_id": f"strategy_{idx:05d}", "strategy_type": s.get("type"), 
                  "strategy_params": str(s)} for idx, s in enumerate(strategies)]
strategy_params_df = spark.createDataFrame(strategy_rows)

# Partition and cache
price_data_partitioned = price_data.repartition(100, "ticker").cache()
backtest_jobs = price_data_partitioned.crossJoin(strategy_params_df)

# Define schema
results_schema = StructType([
    StructField("ticker", StringType()), StructField("strategy_id", StringType()),
    StructField("strategy_type", StringType()), StructField("strategy_params", StringType()),
    StructField("sharpe_ratio", DoubleType()), StructField("max_drawdown", DoubleType()),
    StructField("total_return", DoubleType()), StructField("win_rate", DoubleType()),
    StructField("calmar_ratio", DoubleType()), StructField("num_trades", IntegerType()),
    StructField("avg_trade_return", DoubleType())
])

print(f"Started: {datetime.now().strftime('%H:%M:%S')}")
print(f"\n⏳ Processing {total_backtests:,} backtests...")
print("This will take 30-60 minutes. Kaggle won't timeout! ☕\n")

start_time = datetime.now()

# RUN!
results = backtest_jobs.groupBy("ticker", "strategy_id", "strategy_type", "strategy_params").applyInPandas(
    backtest_udf, schema=results_schema)

strategy_stats = results.groupBy("strategy_id", "strategy_type", "strategy_params").agg(
    avg("sharpe_ratio").alias("avg_sharpe"), stddev("sharpe_ratio").alias("sharpe_std"),
    avg("max_drawdown").alias("avg_drawdown"), avg("total_return").alias("avg_return"),
    avg("win_rate").alias("avg_win_rate"), avg("calmar_ratio").alias("avg_calmar"),
    count("ticker").alias("num_stocks"), spark_min("sharpe_ratio").alias("min_sharpe"),
    spark_max("sharpe_ratio").alias("max_sharpe"))

best_strategies = strategy_stats.filter((col("avg_sharpe") > 0.3) & (col("num_stocks") >= 10)).orderBy(col("avg_sharpe").desc())

duration = (datetime.now() - start_time).total_seconds()

print("\n" + "=" * 80)
print("✅ MAXIMUM SCALE BACKTEST COMPLETE!")
print("=" * 80)
print(f"Duration: {duration/60:.1f} minutes")
print(f"Throughput: {total_backtests/duration:.0f} backtests/second")
print(f"Records processed: {num_records:,}")
print("=" * 80)


🚀 MAXIMUM SCALE PARALLEL BACKTESTING
Processing 12,300 backtests...
This is 6-10x MORE than your Colab run!
Started: 04:12:23

⏳ Processing 12,300 backtests...
This will take 30-60 minutes. Kaggle won't timeout! ☕


✅ MAXIMUM SCALE BACKTEST COMPLETE!
Duration: 0.0 minutes
Throughput: 40496 backtests/second
Records processed: 303,600


In [9]:
# ============================================
# VERIFY RESULTS AND CHECK SCALE
# ============================================

print("🔍 VERIFYING MAXIMUM SCALE RESULTS")
print("=" * 80)

# Force computation by counting results
print("Counting actual results (this forces computation)...\n")

num_results = results.count()
num_stats = strategy_stats.count()
num_best = best_strategies.count()

print(f"✅ Results verified!")
print(f"   Individual results: {num_results:,}")
print(f"   Strategy statistics: {num_stats}")
print(f"   Best strategies: {num_best}")

# Get actual statistics
stats_df = strategy_stats.toPandas()

print(f"\n📊 ACTUAL PERFORMANCE:")
print(f"   Best Sharpe: {stats_df['avg_sharpe'].max():.3f}")
print(f"   Median Sharpe: {stats_df['avg_sharpe'].median():.3f}")
print(f"   Strategies with Sharpe > 0.3: {(stats_df['avg_sharpe'] > 0.3).sum()}")
print(f"   Strategies with Sharpe > 0.5: {(stats_df['avg_sharpe'] > 0.5).sum()}")
print(f"   Strategies with Sharpe > 1.0: {(stats_df['avg_sharpe'] > 1.0).sum()}")

print(f"\n🏆 TOP 10 STRATEGIES:")
print("=" * 80)

best_df = best_strategies.toPandas()

if len(best_df) == 0:
    print("⚠️ No strategies met criteria (Sharpe > 0.3, stocks >= 10)")
    print("Showing all strategies sorted by Sharpe...\n")
    best_df = stats_df.nlargest(10, 'avg_sharpe')

for idx, row in best_df.head(10).iterrows():
    print(f"\n#{idx+1}: {row['strategy_type'].upper()}")
    print(f"   Sharpe: {row['avg_sharpe']:.3f}")
    print(f"   Return: {row['avg_return']*100:.1f}%")
    print(f"   Drawdown: {row['avg_drawdown']*100:.1f}%")
    print(f"   Win Rate: {row['avg_win_rate']*100:.1f}%")
    print(f"   Stocks: {row['num_stocks']}")

print("\n" + "=" * 80)

# Data summary
print(f"\n📊 SCALE ACHIEVED:")
print(f"   Stocks analyzed: {num_tickers}")
print(f"   Strategies tested: {num_stats}")
print(f"   Total backtests: {num_results:,}")
print(f"   Price records: {num_records:,}")

print("=" * 80)

🔍 VERIFYING MAXIMUM SCALE RESULTS
Counting actual results (this forces computation)...



✅ Results verified!
   Individual results: 12,300
   Strategy statistics: 123
   Best strategies: 0



📊 ACTUAL PERFORMANCE:
   Best Sharpe: 0.274
   Median Sharpe: 0.020
   Strategies with Sharpe > 0.3: 0
   Strategies with Sharpe > 0.5: 0
   Strategies with Sharpe > 1.0: 0

🏆 TOP 10 STRATEGIES:


⚠️ No strategies met criteria (Sharpe > 0.3, stocks >= 10)
Showing all strategies sorted by Sharpe...


#121: RSI
   Sharpe: 0.274
   Return: 38.5%
   Drawdown: -32.2%
   Win Rate: 7.7%
   Stocks: 100

#44: RSI
   Sharpe: 0.218
   Return: 31.0%
   Drawdown: -36.2%
   Win Rate: 10.0%
   Stocks: 100

#102: RSI
   Sharpe: 0.216
   Return: 13.7%
   Drawdown: -20.3%
   Win Rate: 1.9%
   Stocks: 100

#61: RSI
   Sharpe: 0.213
   Return: 16.7%
   Drawdown: -26.7%
   Win Rate: 3.8%
   Stocks: 100

#69: RSI
   Sharpe: 0.199
   Return: 37.1%
   Drawdown: -39.6%
   Win Rate: 13.0%
   Stocks: 100

#35: MEAN_REVERSION
   Sharpe: 0.196
   Return: 14.4%
   Drawdown: -19.8%
   Win Rate: 2.0%
   Stocks: 100

#60: RSI
   Sharpe: 0.192
   Return: 16.3%
   Drawdown: -29.2%
   Win Rate: 5.4%
   Stocks: 100

#98: MEAN_REVERSION
   Sharpe: 0.166
   Return: 18.8%
   Drawdown: -29.1%
   Win Rate: 5.5%
   Stocks: 100

#105: RSI
   Sharpe: 0.165
   Return: 35.4%
   Drawdown: -42.7%
   Win Rate: 16.5%
   Stocks: 

In [15]:
# ============================================
# 3D RISK-RETURN-CONSISTENCY PLOT (FIXED)
# ============================================

import plotly.express as px

# Define variables first
stats_df = strategy_stats.toPandas()
num_backtests_total = 12300
num_stocks_tested = 100
num_strategies = len(stats_df)

print("🎨 Creating 3D interactive plot...")

# Prepare data
plot_df = stats_df.copy()
plot_df['Strategy Name'] = plot_df['strategy_type'] + ' - ' + plot_df['strategy_id'].str[-3:]
plot_df['Return %'] = (plot_df['avg_return'] * 100).round(1)
plot_df['Drawdown %'] = (plot_df['avg_drawdown'] * 100).round(1)
plot_df['Sharpe'] = plot_df['avg_sharpe'].round(3)
plot_df['Win Rate %'] = (plot_df['avg_win_rate'] * 100).round(1)

# Create 3D scatter
fig = px.scatter_3d(
    plot_df,
    x='Drawdown %',
    y='Return %',
    z='Sharpe',
    color='strategy_type',
    size='num_stocks',
    hover_data=['Strategy Name', 'Win Rate %'],
    title=f'<b>3D Strategy Analysis: Risk vs Return vs Sharpe</b><br><sup>{num_backtests_total:,} Backtests on {num_stocks_tested} Stocks</sup>',
    labels={'Drawdown %': 'Max Drawdown (%)', 'Return %': 'Total Return (%)', 'Sharpe': 'Sharpe Ratio'},
    color_discrete_sequence=px.colors.qualitative.Set2,
    height=700
)

fig.update_layout(
    scene=dict(
        xaxis=dict(backgroundcolor="rgb(230, 230, 230)", gridcolor="white", title='Max Drawdown (%)'),
        yaxis=dict(backgroundcolor="rgb(230, 230, 230)", gridcolor="white", title='Total Return (%)'),
        zaxis=dict(backgroundcolor="rgb(230, 230, 230)", gridcolor="white", title='Sharpe Ratio'),
    ),
    font=dict(family="Arial", size=12)
)

fig.show()

print("✅ 3D visualization complete!")
print("💡 Rotate the plot by clicking and dragging!")
print("💡 Zoom with scroll wheel!")

🎨 Creating 3D interactive plot...


✅ 3D visualization complete!
💡 Rotate the plot by clicking and dragging!
💡 Zoom with scroll wheel!


In [16]:
# ============================================
# INTERACTIVE TOP STRATEGIES TABLE (FIXED)
# ============================================

import plotly.graph_objects as go

# Define variables
stats_df = strategy_stats.toPandas()
num_backtests_total = 12300
num_stocks_tested = 100

print("📊 Creating interactive strategy table...")

# Get top 15 strategies
top_15 = stats_df.nlargest(15, 'avg_sharpe').reset_index(drop=True)

# Create interactive table with color coding
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>Rank</b>', '<b>Type</b>', '<b>Sharpe</b>', '<b>Return %</b>', 
                '<b>Drawdown %</b>', '<b>Win Rate %</b>', '<b>Calmar</b>', '<b>Stocks</b>'],
        fill_color='steelblue',
        align='center',
        font=dict(color='white', size=14, family='Arial Black'),
        height=40
    ),
    cells=dict(
        values=[
            list(range(1, len(top_15) + 1)),
            top_15['strategy_type'].str.upper(),
            top_15['avg_sharpe'].round(3),
            (top_15['avg_return'] * 100).round(1),
            (top_15['avg_drawdown'] * 100).round(1),
            (top_15['avg_win_rate'] * 100).round(1),
            top_15['avg_calmar'].round(3),
            top_15['num_stocks']
        ],
        fill_color=[['white', 'lightgray'] * 8],
        align='center',
        font=dict(size=13, family='Arial'),
        height=32
    )
)])

fig.update_layout(
    title=f'<b>Top 15 Trading Strategies</b><br><sup>From {num_backtests_total:,} Backtests on {num_stocks_tested} Stocks</sup>',
    title_font_size=20,
    title_x=0.5,
    height=580,
    margin=dict(l=20, r=20, t=100, b=20)
)

fig.show()

print("✅ Interactive table complete!")
print("💾 Perfect for sharing with interviewers!")

📊 Creating interactive strategy table...


✅ Interactive table complete!
💾 Perfect for sharing with interviewers!


In [17]:
# ============================================
# STRATEGY TYPE RADAR CHART (COOL!)
# ============================================

import plotly.graph_objects as go
import numpy as np

# Define variables
stats_df = strategy_stats.toPandas()

print("🎨 Creating radar chart comparison...")

# Get performance by strategy type
type_summary = stats_df.groupby('strategy_type').agg({
    'avg_sharpe': 'mean',
    'avg_return': 'mean',
    'avg_win_rate': 'mean',
    'avg_drawdown': 'mean'
}).reset_index()

# Normalize metrics to 0-1 scale for radar chart
type_summary['sharpe_norm'] = (type_summary['avg_sharpe'] - type_summary['avg_sharpe'].min()) / (type_summary['avg_sharpe'].max() - type_summary['avg_sharpe'].min())
type_summary['return_norm'] = (type_summary['avg_return'] - type_summary['avg_return'].min()) / (type_summary['avg_return'].max() - type_summary['avg_return'].min())
type_summary['winrate_norm'] = (type_summary['avg_win_rate'] - type_summary['avg_win_rate'].min()) / (type_summary['avg_win_rate'].max() - type_summary['avg_win_rate'].min())
type_summary['drawdown_norm'] = 1 - ((type_summary['avg_drawdown'] - type_summary['avg_drawdown'].min()) / (type_summary['avg_drawdown'].max() - type_summary['avg_drawdown'].min()))

# Create radar chart
fig = go.Figure()

categories = ['Sharpe Ratio', 'Returns', 'Win Rate', 'Risk Control']
colors = ['steelblue', 'coral', 'green', 'orange']

for idx, row in type_summary.iterrows():
    values = [
        row['sharpe_norm'],
        row['return_norm'],
        row['winrate_norm'],
        row['drawdown_norm']
    ]
    
    fig.add_trace(go.Scatterpolar(
        r=values + [values[0]],  # Close the polygon
        theta=categories + [categories[0]],
        fill='toself',
        name=row['strategy_type'].upper(),
        line_color=colors[idx % len(colors)],
        opacity=0.6
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 1], showticklabels=False),
        bgcolor="rgb(245, 245, 245)"
    ),
    title=f'<b>Strategy Type Performance Comparison</b><br><sup>Normalized Metrics Across All Tests</sup>',
    title_font_size=18,
    title_x=0.5,
    height=550,
    showlegend=True,
    legend=dict(x=0.85, y=0.95),
    font=dict(family="Arial", size=12)
)

fig.show()

print("✅ Radar chart complete!")
print("🎯 Shows relative strengths of each strategy type!")

🎨 Creating radar chart comparison...


✅ Radar chart complete!
🎯 Shows relative strengths of each strategy type!


In [18]:
# ============================================
# COMPLETE INTERACTIVE DASHBOARD
# ============================================

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Define all variables
stats_df = strategy_stats.toPandas()
num_backtests_total = 12300
num_stocks_tested = 100
num_strategies = 123
num_records = 303600
best_sharpe = stats_df['avg_sharpe'].max()
median_sharpe = stats_df['avg_sharpe'].median()

print("🎨 Creating COMPLETE interactive dashboard...")
print("This combines everything into one beautiful visualization!")

# Create comprehensive dashboard
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=(
        f'Sharpe Distribution ({num_strategies} Strategies)',
        'Top 10 Strategies',
        'Strategy Type Performance',
        'Return Distribution',
        'Win Rate Distribution',
        'Drawdown Analysis',
        'Risk vs Return (Interactive)',
        'Strategy Consistency',
        'Performance Summary'
    ),
    specs=[
        [{"type": "histogram"}, {"type": "bar"}, {"type": "bar"}],
        [{"type": "histogram"}, {"type": "histogram"}, {"type": "histogram"}],
        [{"type": "scatter", "colspan": 2}, None, {"type": "indicator"}]
    ],
    vertical_spacing=0.10,
    horizontal_spacing=0.10,
    row_heights=[0.33, 0.33, 0.34]
)

# Row 1, Col 1: Sharpe Distribution
fig.add_trace(go.Histogram(x=stats_df['avg_sharpe'], nbinsx=30, marker_color='steelblue', 
                            marker_line_color='black', marker_line_width=1, opacity=0.75, name='Sharpe'), 
              row=1, col=1)

# Row 1, Col 2: Top 10
top_10 = stats_df.nlargest(10, 'avg_sharpe')
labels = [f"{r['strategy_type'][:3].upper()}-{i+1}" for i, r in top_10.iterrows()]
fig.add_trace(go.Bar(y=labels, x=top_10['avg_sharpe'], orientation='h', marker_color='steelblue',
                     text=top_10['avg_sharpe'].round(3), textposition='outside'), row=1, col=2)

# Row 1, Col 3: Type Performance
type_perf = stats_df.groupby('strategy_type')['avg_sharpe'].mean().sort_values(ascending=False)
fig.add_trace(go.Bar(x=type_perf.index, y=type_perf.values, marker_color=['steelblue', 'coral', 'green', 'orange'][:len(type_perf)],
                     text=type_perf.values.round(3), textposition='outside'), row=1, col=3)

# Row 2, Col 1: Returns
fig.add_trace(go.Histogram(x=stats_df['avg_return']*100, nbinsx=30, marker_color='green', opacity=0.75), row=2, col=1)

# Row 2, Col 2: Win Rate
fig.add_trace(go.Histogram(x=stats_df['avg_win_rate']*100, nbinsx=30, marker_color='purple', opacity=0.75), row=2, col=2)

# Row 2, Col 3: Drawdown
fig.add_trace(go.Histogram(x=stats_df['avg_drawdown']*100, nbinsx=30, marker_color='coral', opacity=0.75), row=2, col=3)

# Row 3, Col 1-2: Risk vs Return Scatter
fig.add_trace(go.Scatter(
    x=stats_df['avg_drawdown']*100, y=stats_df['avg_sharpe'],
    mode='markers',
    marker=dict(size=10, color=stats_df['num_stocks'], colorscale='Viridis', showscale=True,
                colorbar=dict(title="Stocks", x=0.65, len=0.3), line=dict(width=0.5, color='black')),
    text=[f"{r['strategy_type']}<br>Sharpe: {r['avg_sharpe']:.3f}" for _, r in stats_df.iterrows()],
    hovertemplate='%{text}<extra></extra>'
), row=3, col=1)

# Row 3, Col 3: Summary Indicator
fig.add_trace(go.Indicator(
    mode="number",
    value=best_sharpe,
    title={"text": f"<b>Best Sharpe</b><br><span style='font-size:14px'>{num_backtests_total:,} Backtests</span>"},
    number={'font': {'size': 50, 'color': 'green'}, 'valueformat': '.3f'}
), row=3, col=3)

# Update all axes
fig.update_xaxes(title_text="Sharpe", showgrid=True, gridcolor='lightgray', row=1, col=1)
fig.update_xaxes(title_text="Sharpe", showgrid=True, gridcolor='lightgray', row=1, col=2)
fig.update_xaxes(title_text="Type", showgrid=True, gridcolor='lightgray', row=1, col=3)
fig.update_xaxes(title_text="Return %", showgrid=True, gridcolor='lightgray', row=2, col=1)
fig.update_xaxes(title_text="Win Rate %", showgrid=True, gridcolor='lightgray', row=2, col=2)
fig.update_xaxes(title_text="Drawdown %", showgrid=True, gridcolor='lightgray', row=2, col=3)
fig.update_xaxes(title_text="Max Drawdown %", showgrid=True, gridcolor='lightgray', row=3, col=1)

fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Strategy", row=1, col=2)
fig.update_yaxes(title_text="Avg Sharpe", row=1, col=3)
fig.update_yaxes(title_text="Count", row=2, col=1)
fig.update_yaxes(title_text="Count", row=2, col=2)
fig.update_yaxes(title_text="Count", row=2, col=3)
fig.update_yaxes(title_text="Sharpe Ratio", row=3, col=1)

# Overall layout
fig.update_layout(
    title_text=f"<b>Complete Trading Backtest Analysis Dashboard</b><br>" +
               f"<sup>{num_backtests_total:,} Backtests | {num_stocks_tested} Stocks | {num_records:,} Records | 10 Years</sup>",
    title_font_size=22,
    title_x=0.5,
    height=1100,
    showlegend=False,
    template='plotly_white',
    font=dict(family="Arial", size=11)
)

fig.show()

print("\n✅ Complete interactive dashboard created!")
print("💾 Click camera icon to download as PNG!")
print("🎯 This is your portfolio centerpiece!")

🎨 Creating COMPLETE interactive dashboard...
This combines everything into one beautiful visualization!



✅ Complete interactive dashboard created!
💾 Click camera icon to download as PNG!
🎯 This is your portfolio centerpiece!


In [19]:
# ============================================
# ONE STUNNING PORTFOLIO-READY CHART
# ============================================

import plotly.express as px

# Define variables
stats_df = strategy_stats.toPandas()
num_backtests_total = 12300
num_stocks_tested = 100

print("🎨 Creating ONE stunning portfolio chart...")

# Prepare data
plot_df = stats_df.copy()
plot_df['Return (%)'] = (plot_df['avg_return'] * 100).round(1)
plot_df['Drawdown (%)'] = (plot_df['avg_drawdown'] * 100).round(1)
plot_df['Sharpe Ratio'] = plot_df['avg_sharpe'].round(3)
plot_df['Win Rate (%)'] = (plot_df['avg_win_rate'] * 100).round(1)
plot_df['Strategy Type'] = plot_df['strategy_type'].str.upper()

# Create beautiful bubble chart
fig = px.scatter(
    plot_df,
    x='Drawdown (%)',
    y='Sharpe Ratio',
    size='num_stocks',
    color='Strategy Type',
    hover_data={
        'Strategy Type': True,
        'Sharpe Ratio': ':.3f',
        'Return (%)': ':.1f',
        'Drawdown (%)': ':.1f',
        'Win Rate (%)': ':.1f',
        'num_stocks': True
    },
    title=f'<b>Algorithmic Trading Strategy Performance Analysis</b><br>' + 
          f'<sup>{num_backtests_total:,} Backtests on {num_stocks_tested} Stocks | 303,600 Price Records | 2014-2026</sup>',
    labels={
        'Drawdown (%)': 'Maximum Drawdown (%)',
        'Sharpe Ratio': 'Sharpe Ratio (Risk-Adjusted Return)',
        'Strategy Type': 'Strategy Category'
    },
    color_discrete_sequence=px.colors.qualitative.Bold,
    height=700,
    width=1200
)

# Add target zone
fig.add_shape(
    type="rect",
    x0=-25, x1=0,
    y0=0.2, y1=plot_df['Sharpe Ratio'].max() + 0.05,
    fillcolor="lightgreen",
    opacity=0.15,
    layer="below",
    line=dict(color="green", width=2, dash="dash")
)

fig.add_annotation(
    x=-12, y=plot_df['Sharpe Ratio'].max(),
    text="<b>Target Zone</b><br>Low Risk + Positive Sharpe",
    showarrow=True,
    arrowhead=2,
    font=dict(size=12, color="darkgreen"),
    bgcolor="white",
    bordercolor="green",
    borderwidth=2,
    opacity=0.9
)

fig.update_layout(
    template='plotly_white',
    font=dict(family="Arial", size=13),
    hovermode='closest',
    legend=dict(
        title=dict(text="<b>Strategy Type</b>", font=dict(size=14)),
        x=0.02,
        y=0.98,
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor="black",
        borderwidth=1
    )
)

fig.update_xaxes(showgrid=True, gridcolor='lightgray', zeroline=True, zerolinecolor='red', zerolinewidth=2)
fig.update_yaxes(showgrid=True, gridcolor='lightgray', zeroline=True, zerolinecolor='red', zerolinewidth=1)

fig.show()

print("\n✅ STUNNING visualization complete!")
print("🏆 This is portfolio-quality!")
print("💾 Click camera icon (top right) to download PNG")
print("💡 Or save as HTML for interactive version:")
print("   fig.write_html('trading_backtest_dashboard.html')")

🎨 Creating ONE stunning portfolio chart...



✅ STUNNING visualization complete!
🏆 This is portfolio-quality!
💾 Click camera icon (top right) to download PNG
💡 Or save as HTML for interactive version:
   fig.write_html('trading_backtest_dashboard.html')


In [21]:
# ============================================
# SAVE INTERACTIVE HTML VERSION
# ============================================

print("💾 Saving interactive HTML dashboard...")

# Save the last chart as interactive HTML
fig.write_html('trading_backtest_interactive.html')

print("✅ Saved as: trading_backtest_interactive.html")
print("\n📥 Download this file:")
print("   • Fully interactive")
print("   • Can share with interviewers")
print("   • Embed in portfolio website")
print("   • Professional presentation tool")

# Download file
from IPython.display import FileLink
display(FileLink('trading_backtest_interactive.html'))

💾 Saving interactive HTML dashboard...
✅ Saved as: trading_backtest_interactive.html

📥 Download this file:
   • Fully interactive
   • Can share with interviewers
   • Embed in portfolio website
   • Professional presentation tool


/kaggle/working/trading_backtest_interactive.html